In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv('data_hpo-method-validation_2023-01-30_14-43.csv', encoding="utf_16_le", delimiter=";")

# Quick Info

In [ ]:
#visualize completeted pages
maxpage = df["MAXPAGE"]
maxpage.iloc[1:].apply(pd.to_numeric).hist()

# Auswertung Goals & Methods

In [ ]:
#select only finished data set complete datasets
df_finished = df.where((df["FINISHED"] == "1")  | (df["FINISHED" ]== "Wurde die Befragung abgeschlossen (letzte Seite erreicht)?")).dropna(subset=["FINISHED"])

In [ ]:
# select only answers relating to the first question (Which of the following goals did you try to reach by using HPO?) 
df_go01=df_finished.filter(regex=("^GO01"))

# select only answers relating to the second question (Which of the following goals did you successfully attain by using HPO?)
df_go02=df_finished.filter(regex=("^GO02"))

In [ ]:
df_go01.head()

## Teilnehmer Heatmap

In [ ]:
# create data frame summarizing all answers to the GD01 listing methods, goal, participant id and selection

goal_method = []
for index, column in enumerate(df_go01.columns):
  
  row = {}
  
  #extract the method
  item_description = df_go01[column][0]
  method = re.findall("([^/]+$)", item_description)[0]
  goal = re.findall(":\s(.*):", item_description)[0]
  value = df_go01[column].drop(0).apply(lambda x: 1 if x=='ausgewählt' else 0) 



  if index%6 == 0:
    method = "I did not try to reach the goal"
    goal = re.search(":\s(.*):\s(.*):", item_description)
    goal = goal.group(2)
    value = df_go01[column].drop(0).apply(lambda x: 1 if x== r'I did not try to reach this goal.' else 0)



  for entry in value.iteritems():
    row={"method": method, "goal": goal, "participant": entry[0], "selection": entry[1]}
    goal_method.append(row)

df_goal_method = pd.DataFrame(goal_method)

In [ ]:
df_goal_method.head()

In [ ]:
#plot heatmaps on participant level
for participant in df_goal_method.participant.unique():
  participant_heatmap = df_goal_method.loc[df_goal_method.participant==participant].pivot(index='method', columns='goal', values='selection')
  #plt.figure()
  sns.heatmap(participant_heatmap, annot=True)  
  plt.show()

## Gesamt Heatmap

In [ ]:
# define method to create heatmap counting all occurences of method combinations

def create_2d_matrix(df):

  heatmap = []

  for index, column in enumerate(df.columns):

    row = {}
    
    #extract the method
    item_description = df[column][0]
    method = re.findall("([^/]+$)", item_description)[0]
    goal = re.findall(":\s(.*):", item_description)[0]

    count = df[column].str.contains(r'ausgewählt').sum()


    if index%6 == 0:
      method = "I did not try to reach the goal"
      goal = re.search(":\s(.*):\s(.*):", item_description)
      goal = goal.group(2)
      count = df[column].str.contains(r'I did not try to reach this goal.').sum()


    row={"method": method, "goal": goal, "count": count}


    heatmap.append(row)

  df_heatmap = pd.DataFrame(heatmap)


  return df_heatmap.pivot(index='method', columns='goal', values='count')


### Try

In [ ]:
sns.heatmap(create_2d_matrix(df_go01), annot=True)

### Successful

In [ ]:
sns.heatmap(create_2d_matrix(df_go02), annot=True) 

# Correcting implausible answers

In [ ]:
heatmap_pivot = create_2d_matrix(df_go01)
heatmap_pivot_successful = create_2d_matrix(df_go02) 

In [ ]:
#problem negative values, because go02 is not automatically set to 0 when go01 is disselected
heatmap_pivot.subtract(heatmap_pivot_successful)

In [ ]:
tried=df_go01.where(df_go01=="ausgewählt").replace({'ausgewählt': 1})
tried=tried.fillna(0)
success=df_go02.where(df_go02=="ausgewählt").replace({'ausgewählt': 1})
success=success.fillna(0)

In [ ]:
tried.head()

In [ ]:
#check for unplausible results in which successful is selected but tried is not
res=(tried-success.values)
is_negative = (res < 0).any(axis=1)
is_negative[is_negative == True].index

[16,43,84] werden aussortiert in den Successful Quoten berechnungen. Alternativ könnte man auch die einzelnen Antworten korriegieren

In [ ]:
df_go01_clean = df_go01.drop([16,43,84])

In [ ]:
df_go02_clean = df_go02.drop([16,43,84])

In [ ]:
(create_2d_matrix(df_go02_clean).sum(axis=1)/create_2d_matrix(df_go01_clean).sum(axis=1)).drop('I did not try to reach the goal').plot(kind='bar', title='Percentage of self perceived successfully attained Goal')

In [ ]:
seaborn_success=(create_2d_matrix(df_go02_clean).sum(axis=1)/create_2d_matrix(df_go01_clean).sum(axis=1)).drop('I did not try to reach the goal').copy()

In [ ]:
sns.barplot(seaborn_success.values, seaborn_success.index)

In [ ]:
#Successrate filtered
sns.heatmap(((create_2d_matrix(df_go02_clean))/(create_2d_matrix(df_go01_clean))).drop('I did not try to reach the goal'), annot=True)

In [ ]:
#Successrate unfiltered
sns.heatmap(((create_2d_matrix(df_go02))/(create_2d_matrix(df_go01))).drop('I did not try to reach the goal'), annot=True)

In [ ]:
#absolute filtered heatmap
sns.heatmap(create_2d_matrix(df_go01_clean), annot=True)

# Data From unfinished questionaires

Plot Data from participants who only answered part of the questionaire. No filtering from implausible answers

## Maxpage 3

In [ ]:
#select participants with maxpage 3 (the last entry is there to ensure automatic method extraction)
df_maxpage_3=df.where((df['MAXPAGE']=='3') | (df['MAXPAGE']== 'Letzte Seite, die im Fragebogen bearbeitet wurde')).dropna(subset=['MAXPAGE'])

In [ ]:
df_maxpage_3_go01=df_maxpage_3.filter(regex=("^GO01"))
df_maxpage_3_go02=df_maxpage_3.filter(regex=("^GO02"))

In [ ]:
sns.heatmap(create_2d_matrix(df_maxpage_3_go01), annot=True) 

In [ ]:
sns.heatmap(create_2d_matrix(df_maxpage_3_go02), annot=True) 

## Maxpage 4

In [ ]:
df_maxpage_4=df.where((df['MAXPAGE']=='4') | (df['MAXPAGE']== 'Letzte Seite, die im Fragebogen bearbeitet wurde')).dropna(subset=['MAXPAGE'])

In [ ]:
df_maxpage_4_go01=df_maxpage_4.filter(regex=("^GO01"))
df_maxpage_4_go02=df_maxpage_4.filter(regex=("^GO02"))

In [ ]:
sns.heatmap(create_2d_matrix(df_maxpage_4_go01), annot=True) 

In [ ]:
sns.heatmap(create_2d_matrix(df_maxpage_4_go02), annot=True) 

## All Data

In [ ]:
df_all=df.where((df['MAXPAGE']=='3')| (df['MAXPAGE']=='4') | (df['MAXPAGE']=='5') | (df['MAXPAGE']== 'Letzte Seite, die im Fragebogen bearbeitet wurde')).dropna(subset=['MAXPAGE'])

In [ ]:
df_all_go01=df_all.filter(regex=("^GO01"))
df_all_go02=df_all.filter(regex=("^GO02"))

In [ ]:
sns.heatmap(create_2d_matrix(df_all_go01), annot=True) 

In [ ]:
sns.heatmap(create_2d_matrix(df_all_go02), annot=True) 

In [ ]:
#because not all participants answered question go01 the successrate differs from the one calculated above
all_success = (create_2d_matrix(df_all_go02).sum(axis=1)/create_2d_matrix(df_all_go01).sum(axis=1))
sns.barplot(all_success.values, all_success.index)

# Decision Factors


In [ ]:
map = {'I did not consider this decision factor.': np.NaN ,'Very Low': 1, 'Low': 2, 'Neutral': 3, 'High': 4, 'Very High': 5}

In [ ]:
df_decision_factors = df_finished.replace(map)

In [ ]:
df_decision_factors.filter(regex=("^GD01")).dropna(how='all')

**Decision Factor Frage 23 nicht gestellt worden anscheinend** auch nur 3 mal gewählt worden

In [ ]:
rows=[]
for i in range(1,25):
  
  #23 nicht gestellt worden, zumindest nicht in den variablen enthalten
  if i==23:
    continue

  entry={}  

  for j in range(1,15):

    #to do: use the same filtered df

    goal_method_df = df_decision_factors.filter(regex=(f"^GD{i:02d}")).dropna(how='all')
    description_string = goal_method_df.filter(regex=(f"_{j:02d}")).iloc[0,:].values[0]


    factor_df = goal_method_df.drop(0).filter(regex=(f"_{j:02d}"))
    participant_array = factor_df.index
    value_array = factor_df.values

    question = re.findall("([^/]+):", description_string)[0]
    decision_factor = re.findall(": ([^/]+) \(\(", description_string)[0]

    for index, value in enumerate(value_array):
    
      entry={'Question': question, 'Decision Factor': decision_factor, "Value": float(value), "Participant": int(participant_array[index])}
      rows.append(entry)

decision_factor_results = pd.DataFrame(rows)      

In [ ]:
decision_factor_results.head()

## Percentage I did not consider this decision factor.


In [ ]:
nan_decision=decision_factor_results.groupby('Decision Factor')['Value'].apply(lambda _df: _df.isna().sum())
count_decision=decision_factor_results.groupby('Decision Factor')['Value'].count()
combined=count_decision.add(nan_decision)
percentag_nan=(nan_decision/combined)

In [ ]:
sns.barplot(percentag_nan.values, percentag_nan.index)

## Average importance

In [ ]:
decision_factor_results.groupby(["Decision Factor"]).mean().sort_values('Value')['Value'].plot(kind='bar')

In [ ]:
decision_factor_results.groupby(["Decision Factor"]).mean().nlargest(3,'Value')

In [ ]:
decision_factor_results.groupby(["Decision Factor"]).mean().nsmallest(3,'Value')

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(data=decision_factor_results, x="Value", y="Decision Factor")

In [ ]:
decision_factor_results.groupby(['Participant']).mean().plot(kind='bar')

In [ ]:
decision_factor_results.groupby(["Question"]).mean()['Value'].plot(kind='bar')

## Manual Tuning

In [ ]:
mt_decision_factors=decision_factor_results[decision_factor_results.Question.str.startswith('MT')]

In [ ]:
len(mt_decision_factors)

In [ ]:
mt_decision_factors.groupby(["Decision Factor"]).mean()['Value'].plot(kind='bar')

In [ ]:
mt_decision_factors.groupby(["Decision Factor"]).mean().nlargest(3,'Value').index

In [ ]:
mt_decision_factors.groupby(["Decision Factor"]).mean().nsmallest(3,'Value')

## Bayesian optimization

In [ ]:
bo_decision_factors=decision_factor_results[decision_factor_results.Question.str.startswith('BO')]

In [ ]:
len(bo_decision_factors)

In [ ]:
bo_decision_factors.groupby(["Decision Factor"]).mean()['Value'].plot(kind='bar')

In [ ]:
bo_decision_factors.groupby(["Decision Factor"]).mean().nlargest(3,'Value')

In [ ]:
bo_decision_factors.groupby(["Decision Factor"]).mean().nsmallest(3,'Value')

## Grid Search

In [ ]:
gs_decision_factors=decision_factor_results[decision_factor_results.Question.str.startswith('GS')]

In [ ]:
len(gs_decision_factors)

In [ ]:
gs_decision_factors.groupby(["Decision Factor"]).mean()['Value'].plot(kind='bar')

In [ ]:
gs_decision_factors.groupby(["Decision Factor"]).mean().nlargest(3,'Value').index

In [ ]:
gs_decision_factors.groupby(["Decision Factor"]).mean().nsmallest(3,'Value')

## Random Search

In [ ]:
rs_decision_factors=decision_factor_results[decision_factor_results.Question.str.startswith('RS')]

In [ ]:
len(rs_decision_factors)

In [ ]:
rs_decision_factors.groupby(["Decision Factor"]).mean()['Value'].plot(kind='bar')

In [ ]:
rs_decision_factors.groupby(["Decision Factor"]).mean().nlargest(3,'Value').index

In [ ]:
rs_decision_factors.groupby(["Decision Factor"]).mean().nsmallest(3,'Value')

# Demographics

In [ ]:
df_finished.filter(regex=("^DE[0-9]"))

In [ ]:
for i in range(2, 10):
  print(df_finished.filter(regex=(f"^DE{i:02d}")).value_counts())

# Experience

In [ ]:
experience_map={'<2': 1, '2-4':2, '5-7':3, '8-10':4,'11-15':5, '> 15': 6, 'YearsAIExperience': np.NaN}

In [ ]:
df_decision_factors.filter(regex=("^DE02")).replace(experience_map).head()

In [ ]:
#Join Experience bin of participants on existing decision factor data Frame
decision_factor_results_experience=decision_factor_results.join(df_decision_factors.filter(regex=("^DE02")).replace(experience_map), how ='right', on='Participant')

In [ ]:
decision_factor_results_experience=decision_factor_results_experience.dropna(subset=['Question'])
decision_factor_results_experience.head()

In [ ]:
plt.figure(figsize=(50,20))
sns.boxplot(data=decision_factor_results_experience, x="Value", y="Decision Factor", hue='DE02')

In [ ]:
#Create a bin for the Expirience
decision_factor_results_experience['Erfahrung']=pd.cut(decision_factor_results_experience['DE02'], bins=[0,2,6], labels=['Beginner', 'Professional'])
decision_factor_results_experience['Erfahrung'].value_counts()

In [ ]:
# Add Method Labels to the Data Frame
df_methods=decision_factor_results_experience.copy()
df_methods.loc[df_methods.Question.str.startswith('MT'), 'Method']='Manual Tuning'
df_methods.loc[df_methods.Question.str.startswith('BO'), 'Method']='Bayesian Optimization'
df_methods.loc[df_methods.Question.str.startswith('GS'), 'Method']='Grid Search'
df_methods.loc[df_methods.Question.str.startswith('RS'), 'Method']='Random Search'
df_methods.head()

In [ ]:
plt.figure(figsize=(50,20))
sns.catplot(data=decision_factor_results_experience, x="Value", y="Decision Factor", col='DE02', kind='bar')

In [ ]:
plt.figure(figsize=(50,20))
sns.catplot(data=decision_factor_results_experience, x="Value", y="Decision Factor", col='Erfahrung', kind='bar')

In [ ]:
sns.catplot(data=decision_factor_results_experience, x="Value", y="Decision Factor", kind='bar', height=8.27, aspect=11.7/8.27)

In [ ]:
# Shift data around Neutral
data_minus=df_methods.copy()
data_minus.loc[:,"Value"] -= 3
sns.catplot(data=data_minus, x="Value", y="Decision Factor", kind='bar', height=8.27, aspect=11.7/8.27)

In [ ]:
sns.catplot(data=data_minus, x="Value", y="Decision Factor", hue='Method', kind='bar', height=8.27, aspect=11.7/8.27)

In [ ]:
sns.catplot(data=data_minus, x="Value", y="Decision Factor", hue='Method', kind='bar', height=8.27, aspect=11.7/8.27)

In [ ]:
sns.catplot(data=data_minus, x="Value", y="Decision Factor", hue='Erfahrung', kind='bar', height=8.27, aspect=11.7/8.27)

In [ ]:
sns.catplot(data=data_minus, x="Value", y="Decision Factor", hue='Erfahrung',col='Method', kind='bar', height=8.27, aspect=11.7/8.27)

In [ ]:
sns.catplot(data=data_minus, x="Value", y="Decision Factor", hue='Method',col='Erfahrung', kind='bar', height=8.27, aspect=11.7/8.27)

In [ ]:
sns.catplot(data=data_minus, x="Value", y="Decision Factor", hue='Method', kind='bar', height=8.27, aspect=11.7/8.27)

In [ ]:
plt.figure(figsize=(50,20))
sns.catplot(data=decision_factor_results_experience, x="Value", y="Question", col='Erfahrung', kind='bar')

In [ ]:
#g = sns.FacetGrid(decision_factor_results_experience, col="Erfahrung",  row="Decision Factor")
#g.map_dataframe(sns.histplot, x="Value")

# PHD

In [ ]:
#Add Column for phd
phd_series=df_decision_factors.filter(regex=("^DE04")).apply(lambda x: 'PhD' if x[0]=='PhD' else 'No PhD', axis=1) 
phd_join=pd.DataFrame(phd_series, columns=['PhD'])
df_phd=data_minus.join(phd_join , how ='right', on='Participant')
df_phd.head()

In [ ]:
sns.catplot(data=df_phd, x="Value", y="Decision Factor", hue='PhD', kind='bar', height=8.27, aspect=11.7/8.27)